# Sentimentanalyse und Darstellung georeferenzierter Tweets

### Geo-M-303-100 GIS-Programmierung
####  Niklas Kemm (6060432)

## Einleitung

Immer mehr Menschen haben Zugang zum Internet und mobilen Endgeräten. Entsprechend steigt auch die Zahl der Nutzer Sozialer Medien stetig an. Die so generierten Daten werden oftmals seitens der Betreiber zur Verfügung gestellt. So auch im Falle von Twitter, die gleich mehrere APIs unterhalten. So gibt es die Möglichkeit, kostenfrei Tweets der letzten sieben Tage zu durchsuchen oder in begrentzem Rahmen live mitzulesen. Letztere Möglichkeit soll in dieser Arbeit genutzt werden, um sowohl Daten zu sammeln, als diese auch zu analysieren und visualisieren. Im Sinne geographischer Auswertung ist es cvon Nöten, dass diese einen präzisen Standort mitliefern. Dies ist zwar nur bei einem sehr kleinen Teil aller Tweets der Fall, liefert aber trotzallem viele Daten aufgrund der enormen Verbreitung und Nutzung von Twitter. Die Daten sollen anschließend auf ihr Sentiment untersucht werden, ehe sie graphisch in Form von Karten sichtbar gemacht werden. Das entwickelte Python-Modul soll folglich die Möglichkeit bieten, einen einfachen Zugang zur [__Twitter Streaming API__](https://developer.twitter.com/en/docs/tweets/filter-realtime/overview) und zum Filtern georeferenzierter Tweets zu erhalten und die so gesammelten Daten grundlegend auf ihr Sentiment zu untersuchen, sie graphisch darzustellen, um abschließend Schlüsse aus diesen ziehen zu können, ohne 

## Methodik

Grundlegend teilt sich das Modul auf in 
+ die __Datengewinnung__, 
+ die __Sentimentanalyse__,
+ die __Datenaufbereitung__ und 
+ die __Visualisierung__ der erhobene Daten. 

Für jeden dieser Schritte wurde auf unterschiedliche, bereits existierende Module und Pakete zurückgegriffen, die anschließend in ihren Funktionen erweitert und aufeinander abgestimmt wurden, um bestmöglich gemeinsam zu funktionieren. Die daraus resultierenden Funktionalitäten sind funktionsweise im Modul [`twittersentiment`](twittersentiment.py) abrufbar.

### Datengewinnung / [Tweepy](https://www.tweepy.org/)

Wie bereits weiter oben angemerkt wird im Rahmen dieser Ausarbeitung auf die Twitter Streaming API zur Datengewinnung zurückgegriffen. Diese ermöglicht das Abgreifen von Tweets, die in Echtzeit versandt werden. Sie unterscheidet sich grundsätzlich in eine kostenfreie Variante, mit der etwa 1% aller Tweets abgegriffen werden können, und eine nicht-kostenfreie, die das Abrufen aller Tweets erlaubt. Es wird die kostenfreie Variante verwendet. 

Neben dieser ersten Einschränkung besteht aufgrund der Absicht, die Daten schlussendlich auf einer Karte sichtbar zu machen, zudem eine zweite Einschränkung in Form der freiwilligen Georeferenzierung der Tweets durch den Nutzer selbst. Standardmäßig sind die Kurznachrichten __nicht__ georeferenziert, weshalb erneut nur ein kleiner Teil der zur Verfügung stehenden Daten Verwendung findet.

Um diese Daten zu beziehen wird die Python-Bibliothek `tweepy` genutzt, die einen einfacheren Umgang mit der API erlaubt. Diese liefert die Klasse `StreamListener`, die das streamen der Daten ermöglicht. Sie beinhaltet die beiden Funktionen

+ `on_data(self, data)` und
+ `on_error(self, status)`,

in welchen festgelegt wird, was mit empfangenen Daten oder im Falle eines Fehlers geschehen soll. Diese sind so umgeschrieben, dass beim Empfangen eines Tweets dieser durch das Zerlegen in Einzelteile und Abspeicherns der relevantesten Informationen in seiner Größe verringert wird. Zudem werden hierbei die Funktionen

+ `clean_tweet(text)`,
+ `clean_source(source)` und
+ `get_tweet_sentiment(tweet)`

ausgeführt. 

`clean_tweet(text)` ersetzt mit Hilfe regulärer Asudrücke alle Sonderzeichen, URLs und Emojis aus dem Text des Tweets, ehe er mithilfe von `get_tweet_sentiment(tweet)`analysiert wird (dazu mehr unter __Sentimentanalyse__). `clean_source(source)` sorgt lediglich dafür, dass unterschiedliche Quellen von Tweets ersetzt und vereinfacht werden. So entsteht eine JSON-Datei, die folgende Struktur hat:

```json
{
  "data": [
    {
      "date": "Sat Mar 21 13:42:52 +0000 2020",
      "user_name": "Florida S-Chain",
      "user_id": 188857476,
      "coord_clean": "-80.6675577, 26.6845104",
      "coord_lat": 26.6845104,
      "coord_lon": -80.6675577,
      "text": "Looking to score a job at Follett Corporation? You're in luck. Click the link ...,
      "text_clean": "Looking to score a job at Follett Corporation You re in luck Click the link ...",
      "sentiment": "neutral",
      "source": "CareerArc 2.0",
      "source_clean": "Other"
    },
    {...},
  ]
}
```

All diese Funktionen werden letztlich von der Funtion `get_data(location, auth_file)` aufgerufen, die sowohl die Authentifizierung an der API, als auch das Starten des Streams und das abschließende Speichern der Daten übernimmt. Die so gewonnen Daten werden temporär in einem Dictionary gespeichert, ehe sie bei Beenden des Streams in eine JSON-Datei geschrieben/exportiert werden. Diese befinden sich im Ordner  `data`. Da das Beenden des Streams aber das Unterbrechen des Skiptes, beziehungsweise das Unterbrechen des Kernels voraussetzt (dazu mehr unter __Probleme__) und das Sammeln von Daten je nach Menge lange dauern kann, wird ein _test_dataset_ zur Verfügung gestellt.

### Sentimentanalyse / [Textblob](https://textblob.readthedocs.io/en/dev/)

Eine einfache Sentimentanalyse wird mit Hilfe der Bibliothek `textblob`, die sich ihrerseits auf `nltk` stützt, realisiert. Diese erlaubt eine schnelle Analyse der Polatität eines Textes. Die Polarität reicht dabei von -1 (negative Stimmung) über 0 (neutrale Stimmung) hin zu +1 (positive Stimmung). 

Hierfür wird der Texttteil des Tweets mit Hilfe der Funktion `get_tweet_sentiment(tweet)` zunächst in ein Textblob-Objekt umgewandelt, das anschließend auf die Polarität untersucht wird. Der daraus resultierende numerische Wert wird anschließend entsprechend der obigen Aufteilung klassifiziert.

### Datenaufbereitung / [Geopandas](https://geopandas.org/)

Die Aufbereitung der Daten in Dataframes, die am besten mit `altair`zusammen arbeiten, wird mit Hilfe von `pandas` beziehungsweise `geopandas` realisiert. Insgesamt werden drei Funktionen benötigt, um die verschiedenen Dataframes zu erstellen:

+ `create_dataframe_world(data)`,
+ `create_dataframe_usa(data)`, sowie 
+ `create_dataframe_choropleth(data)`.

Die Unterscheidung zwischen `world` und `usa` wird daher getroffen, als dass `altair` Charts dazu neigen, sehr große Notebooks zu erstellen und so die resultierende Größe - sollte man sich lediglich eine Karte der USA ausgeben wollen - geringer gehalten werden kann. In diesen Funktionen wird die vorher abgespeicherte JSON-Datei als Dataframe mit der Unterteilung der Spalten entsprechend der Keys verfügbar gemacht. Im Falle von `world` reicht dies bereits aus. (<span style="color:red">Rot</span> = essentiell für Visualisierungen, <span style="color:green">Grün</span> = für Tooltip)

| date | <span style="color:green">user_name</span> | user_id | coord_clean | <span style="color:red">coord_lat</span> | <span style="color:red">coord_lon</span> | <span style="color:green">text</span> | text_clean| <span style="color:red">sentiment</span> | source | <span style="color:red">source_clean</span> |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
|2020-01-22 21:41:42+00:00 | DevilsFanForLife | 132726975 | -73.9977, 40.7419 | 40.741900 | -73.997700  | Drinking a Gravity of Surprise by @GrimmAles @... | Drinking a Gravity of Surprise by Beer Run NYC | neutral | Untappd | Other |
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | 

Im Falle von `usa` wird dieser Dataframe anschließend mittels der `geopandas`-Funktion `sjoin` mit den Grenzen der USA (`us-states.geojson`) verschnitten und so verkleinert (und ein _Geo-Dataframe_ ).

|name|<span style="color:red">geometry</span>|index_right| date | <span style="color:green">user_name</span> | user_id | coord_clean | coord_lat | coord_lon | <span style="color:green">text</span> | text_clean | <span style="color:red">sentiment</span> | source | <span style="color:red">source_clean</span> |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
|Alabama |POLYGON ((-87.35930 35.00118, -85.60667 34.984... | 13842 |2020-01-22 23:05:39+00:00 |TMJ - HSV Sales Jobs | 21318419 |-86.8436124, 34.1748208 | 34.174821 | -86.843612 |Click the link in our bio to see currently ope... | Click the link in our bio to see currently ope... |neutral |	CareerArc 2.0 |	Other|
| ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | ... | 

Umfangreicher ist die Aufbereitung des Dataframes, der für die Darstellung der Choroplethen-Karte vorgesehen ist. Hier wird neben der Verschneidung zudem eine Übersetzung der ordinalen Polarität in absolute, numerische Werte vorgenommen, sodass anschließend eine Mittelwerts-Aggregation je Bundesstaat der USA vorgenommen werden kann. 

|geometry | sentiment | index_right | name |
| :---: | :---: | :---: | :---: |
|POLYGON ((-87.35930 35.00118, -85.60667 34.984...| 0.450000 | 0 | Alabama |
| ...| ...| ...| ...|

### Visualisierung / [Altair](https://altair-viz.github.io/)

Die abschließende Visualisierung erfolgt mit `altair`. Insgesamt kann zwischen fünf (beziehungsweise sechs) unterschiedlichen Ansichten gewählt werden:

|           Funktion      |           Darstellung         |
|:------------------------|:----------------------------- |
|`plot_stats_world(data)` |Ansicht der Anzahl Tweets je Quelle weltweit|
|`plot_stats_usa(data)`   |Ansicht der Anzahl Tweets je Quelle der USA |
|`plot_tweets_world(data)`|alle Tweets weltweit                        |
|`plot_tweets_usa(data)`  |alle Tweets der USA                         |
|`plot_choro_usa(data)`   |Choroplethenkarte der gemittelten Sentimente je US-Staat   |
|`plot_all(data)`         |Interaktive Kombination aller obiger Karten |



## Ergebnisse

Mit Hilfe des entwickelten Moduls ist es nun möglichen, mit einem einfachen Funktionsaufruf das Abfangen und Speichern von Tweets innerhalb bestimmter _Bounding Boxes_ zu initiieren. Mit einem weiteren können die Daten anschließend visualisiert und ausgewertet werden.

Zuerst ist es jedoch notwendig, das Paket `rtree` manuell mittels `Anaconda` zu installieren, da dies im Rahmen dieses Notebooks nicht automatisch über [__MyBinder.org__](https://mybinder.org/) in Verbindung mit `requirements.txt` möglich ist, da `rtree` selbst keine entsprechenden wheel-Dateien für die Installation mit `pip` zur Verfügung stellt.

In [ ]:
%%capture
# ^^^unterdrückt Output^^^

# Seperate Installation von 'rtree'
!conda install rtree --yes

Anschließend kann das entwickelte Modul `twittersentiment.py` eingebunden werden.

In [ ]:
import twittersentiment as ts

Für eine einfachere Nutzung bietet es sich an, Bounding Boxes in Variablen zu definieren.

In [ ]:
boundingbox_world = [-180, -90, 180, 90]
boundingbox_usa = [-171.791110603, 18.91619, -66.96466, 71.3577635769]

Nun kann die Funktion `get_data(location)` aufgerufen werden. 

<span style="color:red">__WICHTIG:__</span> Der so gestartete Stream lässt sich nur durch das Beenden des Kernels schließen. Erst dann wird das Abspeichern der Daten initiiert. Anschließend kann wie gewohnt mit dem Ausführen der Zellen fortgefahren werden.

In [ ]:
ts.get_data(boundingbox_usa)

Anschließend können die gesammelten Daten visualisiert werden. Hierei gilt es zu beachten, dass ein einzelner Datenpunkt sehr klein dargestellt wird, um das Anzeigen vieler Daten zu vereinfachen (mehr dazu in __Fazit und Ausblick__). Das Auslassen des Arguments `data` führt zur Anzeige der vorher gesammelten Daten. Sollten zu wenig Datenpunkte gesammelt worden sein, so kann die Funktion `get_data(location)` erneut ausgeführt werden. Entsprechend neue Daten werden der bereits existierenden JSON-Datei angehängt. 

In [ ]:
ts.plot_stats_usa()

In [ ]:
ts.plot_tweets_usa()

In [ ]:
ts.plot_choro_usa()

Alle Karten bieten die Möglichkeit, über einen Tooltip bei Herüberfahren mit der Maus, weitere Daten anzuzeigen.

Wie vermutlich (abhängig von der Dauern des Laufenlassens des Streams) zu sehen ist, benötigt man viele Daten, um ein aussagekrätigeres Ergebnis (insbesondere in der Choroplethenkarte) zu erzielen. Daher wird nun das _test_dataset_ visualisiert. Dieses umfasst etwa 45.000 Tweets. Die Datenaufbereitung dieses kann daher (insbesondere über Binder-Server) einige Zeit in Anspruch nehmen. 

In [ ]:
ts.plot_all("data/test_dataset.json")

Wie man auf der Weltkarte erkennen kann, eignet sich die Sentimentanalyse lediglich für den englischsprachigen Raum - alle anderen Tweets werden als neutral interpretiert. Auch ist die Sentimentanalyse nicht sonderlich umfangreich und auf die Sprache im Internet angepasst. Insbesondere Sarkasmus und andere Gefühle können nicht interpretiert werden. Nichtsdestotrotz lässt sich die Weltkarte nutzen, um Aufschluß über die vorrangig genutzen Plattformen je Land zu gewinnen. So zeigt sich insbesondere, dass Japan sehr stark Foursquare nutzt, während Instagram global genutzt wird. 

Auf US-Bundesstaatenebene zeigt sich, dass sich mit zunehmender Zahl an Tweets eine Populationskarte abzeichnet. Dies ist allerdings aufgrund der großen Verbreitung mobiler Endgeräte auch zu erwarten. Ein Blick auf die durchschnittlichen Sentimente je Bundesstaat macht deutlich, dass keine großen Unterschiede zum Datenaufnahmezeitpunkt zu herrschen scheinten. Den höchsten Wert erreicht Delaware mit 0.575, den niedrigsten Oregon mit 0.14. Um ein besseres Bild zu erhalten ist es daher von Nöten, größere Datensätze zu analysieren.

## Probleme

Natürlich lief die Entwicklung des Moduls nicht problemlos ab. Auf einige dieser Probleme wird im Folgenden (kurz) eingegangen. 

### Generelles

+ Einstellen von Geo-Tagging 

 Twitter hat am [18. Juni 2019](https://twitter.com/TwitterSupport/status/1141039841993355264) bekanntgegeben, dass das Hinzufügen von Standorten nicht weiter unterstützt wird. Somit fällt ein Großteil der Relevanz dieser Arbeit im Sinne der Datenauswertung weg, da ausschließlich über die in der Twitter-App integrierte Kamera oder über Dritt-Apps genaue Standorte einem Tweet angehangen werden können. Dies erklärt auch, weshalb der Anteil an Tweets aus Quellen wie Foursquare und Instagram derart groß ist.
 
 
+ Nutzen Globaler Variablen im Modul

 Leider nutzt das Modul aktuell noch Globale Variablen, um zum einen den Ouput-Dateinamen festzulegen, als auch das Dictionary zu definieren. 

### Tweepy

+ Stoppen des Streams

 Wie bereits weiter oben erwähnt, ist es aktuell nicht möglich, den einmal gestarteten Stream nach einer gewissen Zeit oder bei eintreffen eines bestimmten Ereignisses zu stoppen. Die Überlegung, ob sich dies mit Multithreading oder andersweitig beheben ließe, gab es kurz. Sie wurde jedoch aufgrund der geringen Einschränkungen und des Z
 eitmangels nicht weiter behandelt und kann als to-do betrachtet werden.
 
 
+ Übergeben/Schreiben der Daten

 Aus obigem Problem folgt zudem die Schwierigkeit, gesammelte Daten adäquat zu speichern. Daher wurde auf den Umweg des temporären Abspeicherns in einem Dictionary ausgewichen, welches erst nach Beenden des Streams in eine JSON-Datei geschrieben wird. Hieraus ergibt sich natürlich eine Belastung des Arbeitsspeichers, welche jedoch vernachlässigbar sein sollte (25MB bei ca. 45.000 Tweets). 

### Geopandas

+ `rtree`

 Siehe __Ergebnisse__.
 
 
+ Fehlen der Daten Alaskas

 Während der Verschneidung der Tweets mit den Bundesstaaten der USA gibt es ein Problem mit dem Umgang von Alaska. Während sowohl auf der Weltkarte, als auch auf der normalen Karte der USA, Tweets aus dieser Region richtig verortet werden, so kommt es während der Aggregation der Sentimente je Bundesstaat zum Verlust dieser Daten. Weshalb dies passiert ist ungeklärt.

### Altair

+ Interaktivität auf geographischen Karten

 Leider bietet `altair` aktuell noch nicht die Möglichkeit, geographische Karten interaktiv, also zoom- und bewegbar zu machen. Das Umwandeln der Karten hin zu Scatterplots, mit Längen- und Breitengraden als Achsen bietet diese zwar, führt jedoch auch zum Verlust der Hintergrundkarten. Auch wird ein Bewegen über die Grenzen von +/-90°/180° ermöglicht, was wenig schön ist. Daher wurde eine zweite statische Zoomstufe auf die USA implementiert. Hier gilt es abzuwarten, ob eine bessere Unterstützung geographischer Inhalte seitens `altair` in Zukunft erfolgen wird.

## Fazit und Ausblick

Das entwickelte Modul `twittersentiment.py` ist in der Lage, unkompliziert Tweets über die Twitter API abzugreifen und diese entsprechend aufzubereiten und zu visualisieren. Hierbei wird eine Sentimentanalyse durchgeführt, die eine Einteilung in _positive_, _negative_ und _neutrale_ Tweets ermöglicht. Eine anschließeden Visualisierung ermöglicht die Darstellung der Tweets  eingefärbt nach Sentiment sowohl weltweit, als auch auf US-Bundesstaatenebene. Zusätzlich kann eine Choroplethenkarte, eingefärbt nach den durchschnittlichen Sentimenten je Bundesstaat, erstellt werden. So wird es möglich, grundlegende Aussagen über die Stimmungen in den USA abzuleiten. Weltweit ist diese Analyse aufgrund der Beschränkung der aussagekräftigeren Sentimentanalyse auf die englische Sprache leider (noch) nicht möglich. Zudem reduziert sich die Sinnhaftigkeit dieses Moduls durch das Abschalten des Geo-Taggings seitens Twitter, sodass sich inzwischen eine Analyse der über Twitter geteilten Instagram-Inhalte eher anbietet - wobei an dem Punkt auch direkt mit der Instagram API gearbeitet werden könnte... 

Insgesamt gibt es also noch eine Menge Verbesserungs- , beziehungsweise Ausbaupotenzial. Angefangen mit dem intuitiveren Einarbeiten der Sentimentanalyse, sodass diese einfacher auch auf fremde Datensätze angewandt werden kann, anstatt direkt in der Datengewinnung angewandt zu werden und so eine recht strenge Formatierung fremder Datensätze voraussetzt. Aktuell bietet sich diese Option bestenfalls für das Anzeigen vorher mit `twittersentiment.py` generierter Daten an. Auch gibt es noch viele Möglichkeiten, weitere Details eines Tweets, sei es der Zeitpunkt des Sendens, die User-ID, oder anderes, auszuwerten. Wünschenswert wäre zudem die einfachere Individualisierung der erzeugten Karten. Während diese in gewissem Rahmen nicht besonders aufwändig wären, einzuarbeiten, so ist dies in gegebenem Zeitrahmen nicht passiert. Ein sehr viel umfangreicheres Unterfangen ist das Einarbeiten eines automatischen Stop-Befehls für den geöffnet Daten-Stream, beispielsweise nach erreichen einer gewissen Anzahl an Tweets oder Zeit. Dies würde die Benutzerfreundlichkeit deutlich steigern. 

Schlussendlich bleibt festzuhalten, dass sich die Analyse georeferenzierter Social-Media-Daten grundsätzlich als hilfreich erweisen kann. Leider trifft dies im Zusammenhang mit Tweets inzwischen nicht mehr, beziehnugsweise nur noch in sehr begrenztem Rahmen zu.